# Partial Differential Equations

This notebook demonstrates numerical solutions to **three fundamental classes of second-order partial differential equations (PDEs)** in two spatial dimensions:

## The Three PDE Types:

### 1. **Elliptic PDE** — *Laplace’s Equation*
The Laplace equation models steady-state phenomena such as electrostatic potential or stationary heat distribution:

$$
\nabla^2 u = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = 0
$$

- **Time-independent**: No time evolution; the solution is purely spatial.
- **Boundary-dominated**: The values along the domain boundaries determine the solution throughout the interior.
---

### 2. **Parabolic PDE** — *Heat Equation*
The heat equation models time-dependent diffusion processes:

$$
\frac{\partial u}{\partial t} = \alpha \nabla^2 u = \alpha\left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right)
$$

- **Time-evolving**: The initial condition evolves over time due to thermal diffusion.
- **Smooths gradients**: Sharp features in the initial condition dissipate gradually.
- **Parameter $\alpha$**: Thermal diffusivity controls how quickly the diffusion occurs.
---

### 3. **Hyperbolic PDE** — *Wave Equation*
The wave equation describes the propagation of waves such as sound, light, or water surface disturbances:

$$
\frac{\partial^2 u}{\partial t^2} = c^2 \nabla^2 u = c^2\left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right)
$$

- **Time-evolving**: The solution exhibits oscillatory behavior and wave fronts.
- **Parameter $c$**: Wave speed, which affects how fast waves travel.

---

## In the below example,

1. **Select a PDE type** from the dropdown menu:
   - `Elliptic: Laplace (2D)`
   - `Parabolic: Heat (2D)`
   - `Hyperbolic: Wave (2D)`

2. **Adjust the physical and numerical parameters** using the sliders:
   - Grid size ($n_x$, $n_y$)
   - Diffusivity ($\alpha$), wave speed ($c$), time step ($\Delta t$), final time ($t_{\text{final}}$)
   - Boundary conditions or initial condition type (`Gaussian` or `Sine`)

3. **Observe the solution behavior**:
   - **Laplace**: A static potential field shaped by boundary conditions.
   - **Heat**: Initial heat concentration spreads over time.
   - **Wave**: A pulse or sinusoidal disturbance propagates through the domain.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, Dropdown, Output, VBox, IntSlider, FloatSlider
from IPython.display import display, clear_output
%matplotlib inline

def solve_laplace(nx=50, ny=50, max_iter=2000, tol=1e-4,
                    bc_top=1.0, bc_bottom=0.0, bc_left=0.0, bc_right=0.0):
    u = np.zeros((ny, nx))
    u[0, :] = bc_bottom    # Bottom boundary
    u[-1, :] = bc_top      # Top boundary
    u[:, 0] = bc_left      # Left boundary
    u[:, -1] = bc_right    # Right boundary

    u_new = u.copy()
    for it in range(max_iter):
        u_new[1:-1, 1:-1] = 0.25 * (u[1:-1, :-2] + u[1:-1, 2:] +
                                    u[:-2, 1:-1] + u[2:, 1:-1])
        diff = np.linalg.norm(u_new - u)
        if diff < tol:
            break
        u, u_new = u_new, u.copy()
    return u, it

def plot_laplace_solution(nx, ny, max_iter, tol, bc_top, bc_bottom, bc_left, bc_right):
    u, iters = solve_laplace(nx, ny, max_iter, tol, bc_top, bc_bottom, bc_left, bc_right)
    plt.figure(figsize=(6,5))
    plt.imshow(u, origin='lower', cmap='viridis', extent=[0, nx-1, 0, ny-1])
    plt.colorbar(label='Potential / Temperature')
    plt.title(f"Laplace Equation (Elliptic PDE)\nConverged in {iters} iterations\n"
              f"(High at top: {bc_top}, Low at bottom: {bc_bottom})", fontsize=12)
    plt.xlabel('x (grid nodes)')
    plt.ylabel('y (grid nodes)')
    plt.annotate("High (e.g., heat source)", xy=(nx/2, ny-1), xytext=(nx/2, ny-5),
                 arrowprops=dict(facecolor='white', arrowstyle='->'),
                 color='white', fontsize=10, ha='center')
    plt.annotate("Low (e.g., heat sink)", xy=(nx/2, 0), xytext=(nx/2, 3),
                 arrowprops=dict(facecolor='white', arrowstyle='->'),
                 color='white', fontsize=10, ha='center')
    plt.tight_layout()
    plt.show()

def solve_heat_equation_2d(nx=50, ny=50, Lx=1.0, Ly=1.0, alpha=0.01, dt=1e-4, final_time=0.1, ic_type='Gaussian'):
    dx = Lx/(nx-1)
    dy = Ly/(ny-1)
    x = np.linspace(0, Lx, nx)
    y = np.linspace(0, Ly, ny)
    X, Y = np.meshgrid(x, y)

    # Initial condition:
    if ic_type == 'Gaussian':
        u = np.exp(-100 * ((X - Lx/2)**2 + (Y - Ly/2)**2))
    elif ic_type == 'Sine':
        u = np.sin(np.pi * X / Lx) * np.sin(np.pi * Y / Ly)
    else:
        u = np.sin(np.pi * X / Lx) * np.sin(np.pi * Y / Ly)

    num_steps = int(final_time / dt)
    for _ in range(num_steps):
        u_new = u.copy()
        u_new[1:-1, 1:-1] = u[1:-1, 1:-1] + alpha * dt * (
            (u[2:, 1:-1] - 2*u[1:-1, 1:-1] + u[:-2, 1:-1])/(dx**2) +
            (u[1:-1, 2:] - 2*u[1:-1, 1:-1] + u[1:-1, :-2])/(dy**2)
        )
        u = u_new.copy()
    return X, Y, u

def plot_heat_equation_2d(nx, ny, Lx, Ly, alpha, dt, final_time, ic_type):
    X, Y, u = solve_heat_equation_2d(nx, ny, Lx, Ly, alpha, dt, final_time, ic_type)
    plt.figure(figsize=(7,6))
    cp = plt.contourf(X, Y, u, levels=50, cmap='hot')
    plt.colorbar(cp, label='Temperature')
    plt.title(f"2D Heat Equation (Parabolic PDE)\nTime = {final_time:.3f} s, Diffusivity α = {alpha}\n"
              f"IC: {ic_type}", fontsize=12)
    plt.xlabel('x (position)')
    plt.ylabel('y (position)')
    center_x = Lx/2
    center_y = Ly/2
    plt.annotate("Heat Source / Peak", xy=(center_x, center_y), xytext=(center_x+0.1, center_y+0.1),
                 arrowprops=dict(facecolor='black', arrowstyle='->'),
                 fontsize=10, color='black')
    plt.tight_layout()
    plt.show()


def solve_wave_equation_2d(nx=50, ny=50, Lx=1.0, Ly=1.0, c=1.0, dt=0.001, final_time=0.5, ic_type='Gaussian'):
    dx = Lx/(nx-1)
    dy = Ly/(ny-1)
    x = np.linspace(0, Lx, nx)
    y = np.linspace(0, Ly, ny)
    X, Y = np.meshgrid(x, y)

    # Initial condition:
    if ic_type == 'Gaussian':
        u0 = np.exp(-100 * ((X - Lx/2)**2 + (Y - Ly/2)**2))
    elif ic_type == 'Sine':
        u0 = np.sin(np.pi * X / Lx) * np.sin(np.pi * Y / Ly)
    else:
        u0 = np.sin(np.pi * X / Lx) * np.sin(np.pi * Y / Ly)

    u_old = u0.copy()
    u = u0.copy()

    u[1:-1, 1:-1] = u0[1:-1, 1:-1] + 0.5 * (c * dt)**2 * (
        (u0[2:, 1:-1] - 2*u0[1:-1, 1:-1] + u0[:-2, 1:-1])/(dx**2) +
        (u0[1:-1, 2:] - 2*u0[1:-1, 1:-1] + u0[1:-1, :-2])/(dy**2)
    )

    num_steps = int(final_time / dt)
    for _ in range(1, num_steps):
        u_new = u.copy()
        u_new[1:-1, 1:-1] = (2*u[1:-1, 1:-1] - u_old[1:-1, 1:-1] +
            (c * dt)**2 * (
                (u[2:, 1:-1] - 2*u[1:-1, 1:-1] + u[:-2, 1:-1])/(dx**2) +
                (u[1:-1, 2:] - 2*u[1:-1, 1:-1] + u[1:-1, :-2])/(dy**2)
            ))
        u_old, u = u, u_new.copy()
    return X, Y, u

def plot_wave_equation_2d(nx, ny, Lx, Ly, c, dt, final_time, ic_type):
    X, Y, u = solve_wave_equation_2d(nx, ny, Lx, Ly, c, dt, final_time, ic_type)
    plt.figure(figsize=(7,6))
    cp = plt.contourf(X, Y, u, levels=50, cmap='coolwarm')
    plt.colorbar(cp, label='Displacement')
    plt.title(f"2D Wave Equation (Hyperbolic PDE)\nTime = {final_time:.3f} s, Wave Speed c = {c}\n"
              f"IC: {ic_type}", fontsize=12)
    plt.xlabel('x (position)')
    plt.ylabel('y (position)')
    max_idx = np.unravel_index(np.argmax(np.abs(u)), u.shape)
    plt.annotate("Wave Front", xy=(X[max_idx], Y[max_idx]), xytext=(X[max_idx]+0.1, Y[max_idx]+0.1),
                 arrowprops=dict(facecolor='black', arrowstyle='->'),
                 fontsize=10, color='black')
    plt.tight_layout()
    plt.show()


laplace_widget = interactive(plot_laplace_solution,
         nx=IntSlider(min=20, max=200, step=10, value=50, description='Grid X (nx):'),
         ny=IntSlider(min=20, max=200, step=10, value=50, description='Grid Y (ny):'),
         max_iter=IntSlider(min=100, max=5000, step=100, value=2000, description='Max Iterations:'),
         tol=FloatSlider(min=1e-6, max=1e-2, step=1e-6, value=1e-4, readout_format='.1e', description='Tolerance:'),
         bc_top=FloatSlider(min=0.0, max=5.0, step=0.1, value=1.0, description='Top BC:'),
         bc_bottom=FloatSlider(min=0.0, max=5.0, step=0.1, value=0.0, description='Bottom BC:'),
         bc_left=FloatSlider(min=0.0, max=5.0, step=0.1, value=0.0, description='Left BC:'),
         bc_right=FloatSlider(min=0.0, max=5.0, step=0.1, value=0.0, description='Right BC:')
)

heat2d_widget = interactive(plot_heat_equation_2d,
         nx=IntSlider(min=30, max=200, step=10, value=50, description='Grid X (nx):'),
         ny=IntSlider(min=30, max=200, step=10, value=50, description='Grid Y (ny):'),
         Lx=FloatSlider(min=0.5, max=5.0, step=0.1, value=1.0, description='Domain Length x:'),
         Ly=FloatSlider(min=0.5, max=5.0, step=0.1, value=1.0, description='Domain Length y:'),
         alpha=FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='Diffusivity α:'),
         dt=FloatSlider(min=1e-5, max=1e-3, step=1e-5, value=1e-4, readout_format='.1e', description='Time Step (dt):'),
         final_time=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Final Time (t):'),
         ic_type=Dropdown(options=['Gaussian', 'Sine'], value='Gaussian', description='IC:')
)

wave2d_widget = interactive(plot_wave_equation_2d,
         nx=IntSlider(min=30, max=200, step=10, value=50, description='Grid X (nx):'),
         ny=IntSlider(min=30, max=200, step=10, value=50, description='Grid Y (ny):'),
         Lx=FloatSlider(min=0.5, max=5.0, step=0.1, value=1.0, description='Domain Length x:'),
         Ly=FloatSlider(min=0.5, max=5.0, step=0.1, value=1.0, description='Domain Length y:'),
         c=FloatSlider(min=0.5, max=5.0, step=0.1, value=1.0, description='Wave Speed (c):'),
         dt=FloatSlider(min=1e-4, max=1e-2, step=1e-4, value=0.001, readout_format='.1e', description='Time Step (dt):'),
         final_time=FloatSlider(min=0.01, max=2.0, step=0.01, value=0.5, description='Final Time (t):'),
         ic_type=Dropdown(options=['Gaussian', 'Sine'], value='Sine', description='IC:')
)

pde_type = Dropdown(options=['Elliptic: Laplace (2D)', 'Parabolic: Heat (2D)', 'Hyperbolic: Wave (2D)'],
                    value='Elliptic: Laplace (2D)', description='PDE Type:')

output = Output()

def update_output(change):
    with output:
        clear_output(wait=True)
        if pde_type.value == 'Elliptic: Laplace (2D)':
            display(laplace_widget)
            print("Elliptic PDE (Laplace):\n"
                  "• Represents steady-state phenomena where the solution is time-independent.\n"
                  "• The entire domain is influenced by the boundary conditions; a high boundary can act as a heat source, while a low boundary acts as a heat sink.\n"
                  "• Here, you see a smooth potential or temperature field, which is globally determined by the fixed boundary values.\n"
                  "• Such equations model situations like electrical potential in a region or static temperature distribution in a solid.")
        elif pde_type.value == 'Parabolic: Heat (2D)':
            display(heat2d_widget)
            print("Parabolic PDE (Heat Equation):\n"
                  "• Models time-evolving diffusion processes, where an initial temperature distribution spreads out over time.\n"
                  "• The simulation shows how localized heat gradually diffuses, smoothing out temperature gradients.\n"
                  "• The rate of diffusion is controlled by the diffusivity (α), and the solution continuously changes with time.\n"
                  "• This behavior is typical in thermal conduction, chemical diffusion, or other gradual dispersal phenomena.")
        elif pde_type.value == 'Hyperbolic: Wave (2D)':
            display(wave2d_widget)
            print("Hyperbolic PDE (Wave Equation):\n"
                  "• Models dynamic processes where disturbances propagate with a finite speed.\n"
                  "• The simulation demonstrates traveling wave fronts, which move across the domain over time.\n"
                  "• This type of PDE is used to represent phenomena such as sound waves, electromagnetic waves, or structural vibrations.\n"
                  "• Notice how the wave fronts are distinct and move without immediately smoothing out, reflecting the finite-speed nature of wave propagation.")


pde_type.observe(update_output, names='value')

explanation = (
    "This demonstrates three fundamental classes of partial differential equations in two dimensions:\n"
    "1. Elliptic (Laplace's Equation): Models steady-state phenomena where boundary conditions instantly affect the entire domain.\n"
    "2. Parabolic (Heat Equation): Describes diffusion processes, showing how heat dissipates over time.\n"
    "3. Hyperbolic (Wave Equation): Represents wave propagation with finite speed, such as vibrations or sound waves.\n\n"
    "Use the dropdown menu to select a PDE type. Adjust the interactive sliders to modify parameters and observe "
)

print(explanation)
update_output(None)
display(VBox([pde_type, output]))

This demonstrates three fundamental classes of partial differential equations in two dimensions:
1. Elliptic (Laplace's Equation): Models steady-state phenomena where boundary conditions instantly affect the entire domain.
2. Parabolic (Heat Equation): Describes diffusion processes, showing how heat dissipates over time.
3. Hyperbolic (Wave Equation): Represents wave propagation with finite speed, such as vibrations or sound waves.

Use the dropdown menu to select a PDE type. Adjust the interactive sliders to modify parameters and observe 


## PDE Simulation and Visualization

This code cell implements the solving and visualizing three fundamental types of second-order partial differential equations (PDEs). These include:

1. **Elliptic PDE** — Steady-state potential distribution (Laplace's equation)
2. **Parabolic PDE** — Diffusion (Heat equation)
3. **Hyperbolic PDE** — Wave propagation (Wave equation)


- **Elliptic PDE (Laplace Equation):**
  solves the equation iteratively by updating the interior values using the average of neighboring grid points. It stops when a convergence criterion (based on the difference norm) is met.

- **Parabolic PDE (Heat Equation):**
 simulates the time-dependent diffusion starting from an initial Gaussian distribution. Zero Dirichlet boundary conditions are applied. Time evolution is performed using an explicit time-stepping scheme.

- **Hyperbolic PDE (Wave Equation):**
models the propagation of a wave using finite-difference scheme. The initial condition is a localized Gaussian peak, and zero boundaries are imposed.

### Key Concepts

- **Information Propagation**:
  - *Elliptic PDEs*: Influence propagates instantaneously (infinite speed) due to global coupling via boundary conditions.
  - *Parabolic PDEs*: Influence diffuses outward gradually, modeling irreversible smoothing of features.
  - *Hyperbolic PDEs*: Information travels at finite speed, preserving sharp features like wavefronts.

### In the below tools,

1. **Choose the PDE type** from the dropdown menu.
2. **Use the slider** to scroll through the snapshots.
   - For elliptic: View iteration progress toward convergence.
   - For parabolic: Observe diffusion over time.
   - For hyperbolic: Watch wavefronts propagate and reflect.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, Dropdown, IntSlider, VBox, Output
from IPython.display import display, clear_output
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

def simulate_oxygen_distribution(nx=50, ny=50, max_iter=1000, tol=1e-4, snapshot_count=20):
    u = np.zeros((ny, nx))
    u[-1, :] = 1.0
    u[0, :] = 0.2
    u[:, 0] = 0.5
    u[:, -1] = 0.5
    snapshots = [u.copy()]
    iterations = [0]
    snapshot_indices = np.linspace(0, max_iter, snapshot_count, dtype=int)
    for it in range(1, max_iter):
        u_new = u.copy()
        u_new[1:-1, 1:-1] = 0.25 * (u[1:-1, :-2] + u[1:-1, 2:] + u[:-2, 1:-1] + u[2:, 1:-1])
        diff = np.linalg.norm(u_new - u)
        u = u_new
        if it in snapshot_indices:
            snapshots.append(u.copy())
            iterations.append(it)
        if diff < tol:
            while len(snapshots) < snapshot_count:
                snapshots.append(u.copy())
                iterations.append(it)
            break
    return snapshots, iterations

def simulate_morphogen_gradient_2d(Nx=100, Ny=100, Lx=1.0, Ly=1.0, D=0.01, dt=1e-4, final_time=0.5, snapshot_count=30):
    dx = Lx / (Nx - 1)
    dy = Ly / (Ny - 1)
    x = np.linspace(0, Lx, Nx)
    y = np.linspace(0, Ly, Ny)
    u = np.exp(-100 * ((x[:, None] - Lx / 2)**2 + (y[None, :] - Ly / 2)**2))
    u[0, :] = 0; u[-1, :] = 0
    u[:, 0] = 0; u[:, -1] = 0
    snapshots = [u.copy()]
    times = [0]
    num_steps = int(final_time / dt)
    snapshot_indices = np.linspace(0, num_steps, snapshot_count, dtype=int)
    for step in range(1, num_steps):
        u_new = u.copy()
        u_new[1:-1, 1:-1] = u[1:-1, 1:-1] + D * dt * (
            (u[2:, 1:-1] - 2 * u[1:-1, 1:-1] + u[:-2, 1:-1]) / dx**2 +
            (u[1:-1, 2:] - 2 * u[1:-1, 1:-1] + u[1:-1, :-2]) / dy**2
        )
        u = u_new
        if step in snapshot_indices:
            snapshots.append(u.copy())
            times.append(step * dt)
    return x, y, snapshots, times

def simulate_nerve_impulse_2d(Nx=100, Ny=100, Lx=1.0, Ly=1.0, c=1.0, dt=1e-4, final_time=1, snapshot_count=30):
    dx = Lx / (Nx - 1)
    dy = Ly / (Ny - 1)
    x = np.linspace(0, Lx, Nx)
    y = np.linspace(0, Ly, Ny)
    u0 = np.exp(-100 * ((x[:, None] - Lx / 4)**2 + (y[None, :] - Ly / 2)**2))
    u_old = u0.copy()
    u = u0.copy()
    snapshots = [u.copy()]
    times = [0]
    num_steps = int(final_time / dt)
    snapshot_indices = np.linspace(0, num_steps, snapshot_count, dtype=int)
    for step in range(1, num_steps):
        u_new = u.copy()
        u_new[1:-1, 1:-1] = (2 * u[1:-1, 1:-1] - u_old[1:-1, 1:-1] +
                            (c * dt / dx)**2 * (u[2:, 1:-1] - 2 * u[1:-1, 1:-1] + u[:-2, 1:-1]) +
                            (c * dt / dy)**2 * (u[1:-1, 2:] - 2 * u[1:-1, 1:-1] + u[1:-1, :-2]))
        u_old = u.copy()
        u = u_new.copy()
        if step in snapshot_indices:
            snapshots.append(u.copy())
            times.append(step * dt)
    return x, y, snapshots, times

current_sim = {}

def run_simulation_pde(pde_type):
    if pde_type == 'Elliptic PDE (Steady-State)':
        snapshots, iterations = simulate_oxygen_distribution()
        current_sim['pde'] = pde_type
        current_sim['snapshots'] = snapshots
        current_sim['times'] = iterations
        current_sim['dim'] = '2D'
    elif pde_type == 'Parabolic PDE (Diffusion)':
        x, y, snapshots, times = simulate_morphogen_gradient_2d()
        current_sim['pde'] = pde_type
        current_sim['snapshots'] = snapshots
        current_sim['times'] = times
        current_sim['x'] = x
        current_sim['y'] = y
        current_sim['Lx'] = x[-1]
        current_sim['Ly'] = y[-1]
        current_sim['dim'] = '2D'
        current_sim['Nx'] = x.size
        current_sim['Ny'] = y.size
    elif pde_type == 'Hyperbolic PDE (Wave)':
        x, y, snapshots, times = simulate_nerve_impulse_2d()
        current_sim['pde'] = pde_type
        current_sim['snapshots'] = snapshots
        current_sim['times'] = times
        current_sim['x'] = x
        current_sim['y'] = y
        current_sim['dim'] = '2D'
        current_sim['Nx'] = x.size
        current_sim['Ny'] = y.size


def update_plot_pde(change=None):
    with output_plot:
        clear_output(wait=True)
        idx = snapshot_slider.value
        pde = current_sim.get('pde', 'Elliptic PDE (Steady-State)')
        snapshots = current_sim.get('snapshots', [])
        times = current_sim.get('times', [])

        Nx = current_sim.get('Nx', 1)
        Ny = current_sim.get('Ny', 1)
        Lx = current_sim.get('Lx', 1)
        Ly = current_sim.get('Ly', 1)

        if pde == 'Elliptic PDE (Steady-State)':
            iter_val = times[idx]
            snapshot = snapshots[idx]
            plt.figure(figsize=(7,6))
            plt.imshow(snapshot, origin='lower', cmap='plasma',
                       extent=[0, snapshot.shape[1]-1, 0, snapshot.shape[0]-1])
            plt.colorbar(label='Potential / Temperature')
            title_text = (f"Elliptic PDE (Steady-State)\nIteration: {iter_val}\n"
                          "The steady-state solution is computed from boundary conditions, "
                          "where a high boundary (e.g. heat source) and a low boundary (e.g. heat sink) determine the field globally.")
            plt.title(title_text)
            plt.xlabel('Grid Node x')
            plt.ylabel('Grid Node y')
            nx = snapshot.shape[1]
            ny = snapshot.shape[0]
            plt.annotate('High (Source)', xy=(nx/2, ny-1), xytext=(nx/2, ny-5),
                         arrowprops=dict(facecolor='white', arrowstyle='->'),
                         color='white', fontsize=10, ha='center')
            plt.annotate('Low (Sink)', xy=(nx/2, 0), xytext=(nx/2, 4),
                         arrowprops=dict(facecolor='white', arrowstyle='->'),
                         color='white', fontsize=10, ha='center')
            plt.tight_layout()
            plt.show()
        elif pde == 'Parabolic PDE (Diffusion)':
            x = current_sim.get('x')
            y = current_sim.get('y')
            time_val = times[idx]
            snapshot = snapshots[idx]
            fig, ax = plt.subplots(1, 2, figsize=(14, 6))

            ax[0].plot(x, snapshot[int(Ny/2), :], 'g-', lw=2)
            ax[0].set_title(f"Parabolic PDE (Diffusion) - 1D\nTime: {time_val:.4f} s")
            ax[0].set_xlabel('Position')
            ax[0].set_ylabel('Concentration')
            ax[0].set_ylim(-0.1, 1.1)

            c = ax[1].imshow(snapshot, origin='lower', cmap='plasma',
                             extent=[0, Lx, 0, Ly])
            fig.colorbar(c, ax=ax[1])
            ax[1].set_title(f"Parabolic PDE (Diffusion) - 2D\nTime: {time_val:.4f} s")
            ax[1].set_xlabel('X Position')
            ax[1].set_ylabel('Y Position')

            plt.tight_layout()
            plt.show()
        elif pde == 'Hyperbolic PDE (Wave)':
            x = current_sim.get('x')
            y = current_sim.get('y')
            time_val = times[idx]
            snapshot = snapshots[idx]
            fig, ax = plt.subplots(1, 2, figsize=(14, 6))

            ax[0].plot(x, snapshot[int(Ny/2), :], 'b-', lw=2)
            ax[0].set_title(f"Hyperbolic PDE (Wave) - 1D\nTime: {time_val:.4f} s")
            ax[0].set_xlabel('Position')
            ax[0].set_ylabel('Displacement')
            ax[0].set_ylim(-0.2, 1.2)

            c = ax[1].imshow(snapshot, origin='lower', cmap='viridis',
                             extent=[0, Lx, 0, Ly])
            fig.colorbar(c, ax=ax[1])
            ax[1].set_title(f"Hyperbolic PDE (Wave) - 2D\nTime: {time_val:.4f} s")
            ax[1].set_xlabel('X Position')
            ax[1].set_ylabel('Y Position')

            plt.tight_layout()
            plt.show()


pde_type = Dropdown(options=['Elliptic PDE (Steady-State)', 'Parabolic PDE (Diffusion)', 'Hyperbolic PDE (Wave)'],
                    value='Elliptic PDE (Steady-State)', description='PDE Type:')
snapshot_slider = IntSlider(min=0, max=1, step=1, value=0, description='Snapshot:')
output_plot = Output()

def on_pde_change(change):
    run_simulation_pde(pde_type.value)
    snapshots = current_sim.get('snapshots', [])
    snapshot_slider.max = len(snapshots) - 1
    snapshot_slider.value = 0
    update_plot_pde()

pde_type.observe(on_pde_change, names='value')
snapshot_slider.observe(update_plot_pde, names='value')

run_simulation_pde(pde_type.value)
snapshot_slider.max = len(current_sim.get('snapshots', [])) - 1

explanation = (
    "How information propagates?\n"
    "This tool demonstrates how solutions of different PDE types evolve through iterative processes:\n"
    "1. Elliptic PDE (Steady-State): The solution is computed directly from boundary conditions, so the entire domain is influenced instantly.\n"
    "2. Parabolic PDE (Diffusion): An initial condition diffuses over time, gradually smoothing out sharp gradients.\n"
    "3. Hyperbolic PDE (Wave): Disturbances propagate as waves with finite speed, maintaining distinct wave fronts.\n\n"
    "Use the dropdown to select a PDE type and the slider to view different iterations or time snapshots."
)
print(explanation)
display(VBox([pde_type, snapshot_slider, output_plot]))
update_plot_pde()


How information propagates?
This tool demonstrates how solutions of different PDE types evolve through iterative processes:
1. Elliptic PDE (Steady-State): The solution is computed directly from boundary conditions, so the entire domain is influenced instantly.
2. Parabolic PDE (Diffusion): An initial condition diffuses over time, gradually smoothing out sharp gradients.
3. Hyperbolic PDE (Wave): Disturbances propagate as waves with finite speed, maintaining distinct wave fronts.

Use the dropdown to select a PDE type and the slider to view different iterations or time snapshots.
